In [1]:
import xarray as xr
import pandas as pd
import pysumma as ps

def convert_time_to_summa_string(t):
    return (
        f'{t.dt.year.values[()]:04}'
        f'-{t.dt.month.values[()]:02}'
        f'-{t.dt.day.values[()]:02}'
        f' {t.dt.hour.values[()]:02}'
        f':{t.dt.minute.values[()]:02}'
    )

attrs = {
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}
}
name_lookup = {
    'airpres':  'pressure',
    'airtemp':  'temperature',
    'spechum':  'specific_humidity',
    'windspd':  'windspeed',
    'SWRadAtm': 'shortwave_radiation',
    'LWRadAtm': 'longwave_radiation',
    'pptrate':  'total_precipitation',
}

In [2]:
df = pd.read_csv('../NLDAS2_WY22.csv')
df.index = pd.DatetimeIndex(df['time_convert'], name='time')

forcing_filename = 'hj_andrews_wy2021.nc'
# Adding 1 hour to account for SUMMA being period-ending
time_idx = df.index + pd.Timedelta('1H')
shape = (len(time_idx), 1, )
dims = ('time', 'hru', )
coords = {'time': time_idx}

met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 2021-10-01'
for varname, varattrs in attrs.items():
    df_name = name_lookup[varname]
    met_data[varname] = xr.DataArray(
        data=df[df_name].values.reshape(-1, 1),
        coords=coords, dims=dims, name=varname, attrs=varattrs
    )

met_data['airtemp'] += 273.16  # Convert to Kelvin
met_data['pptrate'] /= 3600.0  # Convert to mm/s
met_data['data_step'] = xr.Variable([], 3600.0)
met_data.to_netcdf(f'./forcings/{forcing_filename}')

with open('./forcings/forcing_file_list.txt', 'w') as f:
    f.write(f"'{forcing_filename}'\n")

In [3]:
lat = 44.2072180256268
lon = -122.119450090239
elev = 1050
local_attrs = xr.open_dataset('../summa_setup_template/params/local_attributes.nc').load()
local_attrs['longitude'].values[:] = lon
local_attrs['latitude'].values[:] = lat
local_attrs['elevation'].values[:] = elev
# 1 is evergreen needleleaf forest in the MODIFIED_IGBP_MODIS_NOAH option
# for the `vegeParTbl` decision. This can be found in the `VEGPARM.TBL` file
local_attrs['vegTypeIndex'].values[:] = 1
local_attrs.to_netcdf('./params/local_attributes.nc')

In [4]:
!./install_local_setup.sh

summa_exe = '/home/bzq/workspace/summa/bin/summa.exe'
file_manager = './file_manager.txt'
s = ps.Simulation(summa_exe, file_manager)

t0 = met_data['time'].isel(time=0)
t1 = met_data['time'].isel(time=-1)
s.manager['simStartTime'] = convert_time_to_summa_string(t0)
s.manager['simEndTime'] = convert_time_to_summa_string(t1)

s.run('local')
s.status

'Success'